In [5]:
#TO GET ALL THE GAMES IN A SEASON
from nba_api.stats.endpoints import leaguegamefinder
from sympy.codegen.ast import continue_

#Get regular season games
gamefinder = leaguegamefinder.LeagueGameFinder(
    season_nullable='2025-26',
    league_id_nullable='00',
    season_type_nullable='Regular Season'
)
games = gamefinder.get_data_frames()[0]
game_ids = games['GAME_ID'].unique().tolist()
len(game_ids)

611

In [13]:
game_ids

['0022500583',
 '0022500582',
 '0022500585',
 '0022500586',
 '0022500581',
 '0022500580',
 '0022500579',
 '0022500584',
 '0022500578',
 '0022500576',
 '0022500577',
 '0022500574',
 '0022500572',
 '0022500575',
 '0022500573',
 '0022500571',
 '0022500565',
 '0022500566',
 '0022500567',
 '0022500569',
 '0022500570',
 '0022500568',
 '0022500564',
 '0022500559',
 '0022500558',
 '0022500561',
 '0022500563',
 '0022500560',
 '0022500562',
 '0022500553',
 '0022500557',
 '0022500556',
 '0022500548',
 '0022500551',
 '0022500550',
 '0022500549',
 '0022500552',
 '0022500555',
 '0022500554',
 '0022500544',
 '0022500546',
 '0022500542',
 '0022500543',
 '0022500547',
 '0022500545',
 '0022500532',
 '0022500534',
 '0022500540',
 '0022500541',
 '0022500537',
 '0022500535',
 '0022500533',
 '0022500539',
 '0022500538',
 '0022500536',
 '0022500530',
 '0022500528',
 '0022500531',
 '0022500524',
 '0022500523',
 '0022500518',
 '0022500521',
 '0022500525',
 '0022500516',
 '0022500519',
 '0022500522',
 '00225005

In [36]:
from nba_api.stats.endpoints import BoxScoreTraditionalV3
from nba_api.stats.endpoints import playbyplayv3
from nba_api.stats.endpoints import leaguegamefinder
import time
import random
from requests.exceptions import ReadTimeout, ConnectionError, RequestException, Timeout
from json.decoder import JSONDecodeError
import re
import pandas as pd
def is_int_convertible(x):
    try:
        int(x)
        return True
    except (ValueError, TypeError):
        return False

def get_minutes_from_v3_clock(clock_str):
    if not isinstance(clock_str, str): return 0

    # Check for PTxxM format
    match = re.search(r'PT(\d+)M', clock_str)
    if match:
        return int(match.group(1))

    # Check for MM:SS format
    if ':' in clock_str:
        return int(clock_str.split(':')[0])

    return 0


gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2025-26',
                                              league_id_nullable='00',
                                              season_type_nullable='Regular Season')
games = gamefinder.get_data_frames()[0]
pbp = playbyplayv3.PlayByPlayV3("0022500544").get_data_frames()[0]
stats_df = BoxScoreTraditionalV3(game_id="0022500544").get_data_frames()[0]

game_dict = {}
game="0022500544"

try:
    game_dict['GAME_ID']=game
    game_dict['TEAM1']= str(games[games["GAME_ID"]== game].iloc[0]["TEAM_ABBREVIATION"])
    game_dict['TEAM2']= str(games[games["GAME_ID"]== game].iloc[1]["TEAM_ABBREVIATION"])
    game_dict['DATE']= str(games[games["GAME_ID"]== game].iloc[0]["GAME_DATE"])
    game_dict['PF']= games[games["GAME_ID"]== game].iloc[0]["PF"] + games[games["GAME_ID"]== game].iloc[1]["PF"]

    # --- A. Clutch Calculation ---
    period = pbp['period'].unique().max()
    if period == 5:
        game_dict['CLUTCH'] = 0.95
    elif period >= 6:
        game_dict['CLUTCH'] = 1.0
    else:
#           Filter for 4th quarter +
        pbp_valid = pbp[pbp['period'] >= 4].copy()

        # Filter for last 5 minutes using helper (We apply the function to the 'clock' column)
        pbp_valid['MINUTES_REMAINING'] = pbp_valid['clock'].apply(get_minutes_from_v3_clock)
        pbp_valid = pbp_valid[pbp_valid['MINUTES_REMAINING'] < 5]

        # Calculate Margin Manually (Home - Away), Ensure we treat None/NaN as 0 or forward fill if needed
        pbp_valid['scoreHome'] = pd.to_numeric(pbp_valid['scoreHome'], errors='coerce')
        pbp_valid['scoreAway'] = pd.to_numeric(pbp_valid['scoreAway'], errors='coerce')

        pbp_valid[['scoreHome', 'scoreAway']] = pbp_valid[['scoreHome', 'scoreAway']].ffill()
        pbp_valid = pbp_valid.dropna(subset=['scoreHome', 'scoreAway'])

        pbp_valid['margin'] = pbp_valid['scoreHome'] - pbp_valid['scoreAway']


        # Count possessions within +/- 5 points
        clutch_mask = (pbp_valid['margin'] <= 5) & (pbp_valid['margin'] >= -5)
        clutch_count = clutch_mask.sum()

        game_dict['CLUTCH'] = clutch_count

    # --- B. Performance Calculation ---
    if stats_df is None:
        print(f"  [x] Failed to fetch BoxScore for {game}. Skipping.")


    stats_df['GAME_SCORE'] = (
        stats_df['points'] + 0.4 * stats_df['fieldGoalsMade'] - 0.7 * stats_df['fieldGoalsAttempted'] -
        0.4 * (stats_df['freeThrowsAttempted'] - stats_df['freeThrowsMade']) + 0.7 * stats_df['reboundsOffensive'] +
        0.3 * stats_df['reboundsDefensive'] + stats_df['steals'] + 0.7 * stats_df['assists'] +
        0.7 * stats_df['blocks'] - 0.4 * stats_df['foulsPersonal'] - stats_df['turnovers']
    )

    performance_sum = 0.0
    for gs in stats_df['GAME_SCORE']:
        if gs >= 70.0: performance_sum += 0.95
        elif gs >= 60.0: performance_sum += 0.9
        elif gs >= 50.0: performance_sum += 0.8
        elif gs >= 40.0: performance_sum += 0.7

    game_dict['PERFORMANCE'] = performance_sum


    # --- C. Highlights Calculation ---
    pbp['description'] = pbp['description'].astype(str).str.upper()

    # Dunks: look for 'DUNK' in the single description column
    # Filter out 'MISS' to be safe, though usually 'Dunk' implies made in text
    if 'shotResult' in pbp.columns:
        dunks = pbp[
            (pbp['description'].str.contains("DUNK", case=False)) &
            (pbp['shotResult'] == 'Made')
        ].shape[0]
    else:
        # Fallback if specific column is missing
        dunks = pbp[
            (pbp['description'].str.contains("DUNK", case=False)) &
            (~pbp['description'].str.contains("MISS", case=False))
        ].shape[0]

    # Fix for threes: ensure we grab values safely
    threes= int(games[games["GAME_ID"]== game].iloc[0]["FG3M"] + games[games["GAME_ID"]== game].iloc[1]["FG3M"])

    blocks = pbp[pbp['description'].str.contains("BLOCK")].shape[0]

    #calculate highlights/minutes
    total_highlights = dunks + (0.75 * threes) + blocks
    highlights_density = total_highlights / 48

    game_dict['HIGHLIGHTS'] = highlights_density

    # Standard sleep
    time.sleep(random.uniform(0.5, 1.0))

except Exception as e:
    print(f"  [!!!] Critical Error processing game {game}: {e}")

print(clutch_mask)
stats_df["GAME_SCORE"]
print(dunks)
print(threes)
print(blocks)


473    False
474     True
481     True
486     True
491     True
500     True
503     True
504     True
505     True
509     True
515     True
517     True
519     True
540     True
554     True
555    False
559    False
Name: margin, dtype: bool
12
19
19


In [47]:
#How to get play by play data
from nba_api.stats.endpoints import playbyplayv3
pbp = playbyplayv3.PlayByPlayV3("0022500332")
pbp = pbp.get_data_frames()[0]
#
# player_stat_data = BoxScoreTraditionalV2(game_id='0022401161')
# stats_df = player_stat_data.player_stats.get_data_frame()
pbp

,gameId,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,...,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId
0,0022500332,2,PT12M00.00S,1,0,,0,,,0,...,0,0,0,,Start of 1st Period (8:41 PM EST),period,start,1,0,1
1,0022500332,7,PT12M00.00S,1,1610612742,DAL,203076,Davis,A. Davis,0,...,,,0,h,Jump Ball Davis vs. Adebayo: Tip to Herro,Jump Ball,,1,0,2
2,0022500332,10,PT11M43.00S,1,1610612748,MIA,1628389,Adebayo,B. Adebayo,5,...,0,3,3,v,Adebayo 26' 3PT Jump Shot (3 PTS) (Wiggins 1 AST),Made Shot,Jump Shot,1,3,3
3,0022500332,12,PT11M16.00S,1,1610612742,DAL,1642948,Nembhard,R. Nembhard,-132,...,,,0,h,MISS Nembhard 13' Step Back Jump Shot,Missed Shot,Step Back Jump shot,1,2,4
4,0022500332,13,PT11M13.00S,1,1610612748,MIA,203952,Wiggins,A. Wiggins,0,...,,,0,v,Wiggins REBOUND (Off:0 Def:1),Rebound,Unknown,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,0022500332,673,PT00M24.20S,4,1610612742,DAL,1642948,Nembhard,R. Nembhard,-67,...,,,0,h,MISS Nembhard 21' Step Back Jump Shot,Missed Shot,Step Back Jump shot,1,2,484
484,0022500332,674,PT00M20.80S,4,1610612748,MIA,1628389,Adebayo,B. Adebayo,0,...,,,0,v,Adebayo REBOUND (Off:1 Def:6),Rebound,Unknown,1,0,485
485,0022500332,675,PT00M08.80S,4,1610612748,MIA,1628389,Adebayo,B. Adebayo,-195,...,,,0,v,MISS Adebayo 27' 3PT Pullup Jump Shot,Missed Shot,Pullup Jump shot,1,3,486
486,0022500332,676,PT00M05.20S,4,1610612742,DAL,1642948,Nembhard,R. Nembhard,0,...,,,0,h,Nembhard REBOUND (Off:0 Def:5),Rebound,Unknown,1,0,487


In [ ]:
##.............Game Data Ingestion Loop.........##
maxClutch=0
maxHighlight=0
maxPerformance=0
for game in game_ids:#[939:1230]:
    dict={}

#Get game id
    dict['GAME_ID']=game

#Get teams, date, fouls
    dict['TEAM1']= str(games[games["GAME_ID"]== game].iloc[0]["TEAM_ABBREVIATION"])
    dict['TEAM2']= str(games[games["GAME_ID"]== game].iloc[1]["TEAM_ABBREVIATION"])
    dict['DATE']= str(games[games["GAME_ID"]== game].iloc[0]["GAME_DATE"])
    dict['PF']= games[games["GAME_ID"]== game].iloc[0]["PF"] + games[games["GAME_ID"]== game].iloc[1]["PF"]

#Extract PBP data
    pbp = playbyplayv2.PlayByPlayV2(game)
    pbp = pbp.get_data_frames()[0]

#Clutch
    period= pbp['PERIOD'].unique().max()
    if period==5:
        dict['CLUTCH']= 0.95
    elif period>=6:
        dict['CLUTCH']= 1.0
    else:
        #filtering plays that happened in 4th quarter or later, with 5 minutes or less left, with Score margin is not missing, and where score margin can be convertable to an integer
        pbp_valid= pbp[(pbp["PERIOD"]>=4) & (pbp["PCTIMESTRING"].str.split(":").str[0].astype(int) < 5) & (pbp['SCOREMARGIN'].notna()) & (pbp['SCOREMARGIN'].apply(is_int_convertible))]

        #count the number of clutch possessions
        clutch=(pbp_valid["SCOREMARGIN"].astype(int) <= 5) & (pbp_valid["SCOREMARGIN"].astype(int) >= -5)
        clutch_count= (clutch==True).sum()

        dict['CLUTCH']= clutch_count
        if clutch_count>maxClutch:
             maxClutch= clutch_count

#Performance
    #Extract boxscore data
    player_stat_data = BoxScoreTraditionalV2(game)
    stats_df = player_stat_data.player_stats.get_data_frame()

    stats_df['GAME_SCORE'] = (
            stats_df['PTS'] + 0.4 * stats_df['FGM'] - 0.7 * stats_df['FGA'] -
            0.4 * (stats_df['FTA'] - stats_df['FTM']) + 0.7 * stats_df['OREB'] +
            0.3 * stats_df['DREB'] + stats_df['STL'] + 0.7 * stats_df['AST'] +
            0.7 * stats_df['BLK'] - 0.4 * stats_df['PF'] - stats_df['TOV']
        )

    performance_sum=0.0
    for gs in stats_df['GAME_SCORE']:
        if gs>=70.0:
            performance_sum+=0.95
        elif gs>=60.0:
            performance_sum+=0.9
        elif gs>=50.0:
            performance_sum+=0.8
        elif gs>=40.0:
            performance_sum+=0.7

    dict['PERFORMANCE']= performance_sum
    if performance_sum>maxPerformance:
         maxPerformance= performance_sum


#Highlights
    dunks= pbp[pbp["HOMEDESCRIPTION"].str.contains("Dunk", case=False, na=False) & ~pbp["HOMEDESCRIPTION"].str.contains("MISS", case=False, na=False) |pbp["VISITORDESCRIPTION"].str.contains("Dunk", case=False, na=False) & ~pbp["VISITORDESCRIPTION"].str.contains("MISS", case=False, na=False) ].shape[0]

    threes= games[games["GAME_ID"]== game].iloc[0]["FG3M"] + games[games["GAME_ID"]== game].iloc[1]["FG3M"]

    blocks= pbp[pbp["HOMEDESCRIPTION"].str.contains("BLOCK", case=False, na=False) | pbp["VISITORDESCRIPTION"].str.contains("BLOCK", case=False, na=False)].shape[0]

    total_highlights = len(dunks) + 0.75*len(threes) + len(blocks)
    highlights_density = total_highlights / 48 #highlights per min
    dict['HIGHLIGHTS']= highlights_density
    if highlights_density>maxHighlight:
         maxHighlight= highlights_density


    df_list.append(dict)

            # Respectful delay to avoid rate limiting
    time.sleep(random.uniform(0.8, 1.5))


In [4]:
from nba_api.stats.endpoints import BoxScoreSummaryV2
from nba_api.stats.endpoints import BoxScoreTraditionalV3

boxscore = BoxScoreTraditionalV3(game_id='0022500583')
team_stats = boxscore.get_data_frames()[1]
team_stats

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,minutes,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,startersBench
0,0022500583,1610612742,Dallas,Mavericks,DAL,mavericks,126:29,30,50,0.600,...,2,19,21,18,8,3,7,12,79,Starters
1,0022500583,1610612742,Dallas,Mavericks,DAL,mavericks,126:29,25,47,0.532,...,7,18,25,19,5,4,6,7,65,Bench
2,0022500583,1610612762,Utah,Jazz,UTA,jazz,139:00,25,57,0.439,...,9,14,23,18,6,2,13,10,63,Starters
3,0022500583,1610612762,Utah,Jazz,UTA,jazz,139:00,22,39,0.564,...,3,13,16,15,3,2,4,4,59,Bench


In [6]:
from nba_api.stats.endpoints import PlayerGameLog

gamelog = PlayerGameLog(
    player_id=1630162,
    season='2025-26',
    season_type_all_star='Regular Season'
)

df_pg = gamelog.get_data_frames()[0]
df_pg

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22025,1630162,0022500598,"Jan 17, 2026",MIN @ SAS,L,40,19,33,0.576,...,4,4,3,0,0,2,4,55,8,1
1,22025,1630162,0022500552,"Jan 11, 2026",MIN vs. SAS,W,38,10,21,0.476,...,2,2,3,0,0,0,2,23,-4,1
2,22025,1630162,0022500542,"Jan 10, 2026",MIN @ CLE,L,35,10,24,0.417,...,4,5,3,0,2,1,1,25,-16,1
3,22025,1630162,0022500530,"Jan 08, 2026",MIN vs. CLE,W,37,10,20,0.500,...,7,7,9,1,1,4,3,25,19,1
4,22025,1630162,0022500513,"Jan 06, 2026",MIN vs. MIA,W,29,8,18,0.444,...,5,5,3,1,2,3,4,26,16,1
5,22025,1630162,0022500498,"Jan 04, 2026",MIN @ WAS,W,30,14,21,0.667,...,6,6,3,4,0,4,2,35,29,1
6,22025,1630162,0022500486,"Jan 03, 2026",MIN @ MIA,W,38,10,19,0.526,...,3,3,5,1,0,2,0,33,-2,1
7,22025,1630162,0022500463,"Dec 31, 2025",MIN @ ATL,L,33,10,18,0.556,...,4,5,2,1,2,2,0,30,-21,1
8,22025,1630162,0022500452,"Dec 29, 2025",MIN @ CHI,W,32,9,13,0.692,...,2,2,4,1,1,0,0,23,28,1
9,22025,1630162,0022500439,"Dec 27, 2025",MIN vs. BKN,L,35,10,22,0.455,...,7,7,2,1,0,3,1,28,-14,1


In [8]:
from nba_api.stats.endpoints import scheduleleaguev2

# Fetch the schedule for the 2024-25 season
# Note: You can change the season parameter to get historical data
schedule = scheduleleaguev2.ScheduleLeagueV2(season='2025-26')
df_schedule = schedule.get_data_frames()[0]

# Inspect the columns (usually contains 'GAME_DATE', 'GAME_TIME', 'GAME_STATUS_TEXT')
# GAME_STATUS_TEXT usually holds the time like "7:30 pm ET"
df_schedule

,leagueId,seasonYear,gameDate,gameId,gameCode,gameStatus,gameStatusText,gameSequence,gameDateEst,gameTimeEst,...,awayOttBroadcasters_broadcasterMedia,awayOttBroadcasters_broadcasterId,awayOttBroadcasters_broadcasterDisplay,awayOttBroadcasters_broadcasterAbbreviation,awayOttBroadcasters_broadcasterDescription,awayOttBroadcasters_tapeDelayComments,awayOttBroadcasters_broadcasterVideoLink,awayOttBroadcasters_broadcasterTeamId,awayOttBroadcasters_broadcasterRanking,awayOttBroadcasters_localizationRegion
0,00,2025-26,10/02/2025 00:00:00,0012500008,20251002/PHINYK,3,Final,1,2025-10-02T00:00:00Z,1900-01-01T12:00:00Z,...,None,NaN,None,None,None,None,None,NaN,None,None
1,00,2025-26,10/03/2025 00:00:00,0012500001,20251003/PHXLAL,3,Final,2,2025-10-03T00:00:00Z,1900-01-01T22:00:00Z,...,ott,1001033.0,Suns+,DTC-PHX,,,,1.610613e+09,None,
2,00,2025-26,10/03/2025 00:00:00,0012500009,20251003/MELNOP,3,Final,1,2025-10-03T00:00:00Z,1900-01-01T05:30:00Z,...,None,NaN,None,None,None,None,None,NaN,None,None
3,00,2025-26,10/04/2025 00:00:00,0012500010,20251004/NYKPHI,3,Final,1,2025-10-04T00:00:00Z,1900-01-01T11:00:00Z,...,None,NaN,None,None,None,None,None,NaN,None,None
4,00,2025-26,10/04/2025 00:00:00,0012500011,20251004/SEMNOP,3,Final,5,2025-10-04T00:00:00Z,1900-01-01T23:00:00Z,...,None,NaN,None,None,None,None,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,00,2025-26,06/08/2026 00:00:00,0042500403,,1,8:30 pm ET,1,2026-06-08T00:00:00Z,1900-01-01T20:30:00Z,...,None,NaN,None,None,None,None,None,NaN,None,None
1312,00,2025-26,06/10/2026 00:00:00,0042500404,,1,8:30 pm ET,1,2026-06-10T00:00:00Z,1900-01-01T20:30:00Z,...,None,NaN,None,None,None,None,None,NaN,None,None
1313,00,2025-26,06/13/2026 00:00:00,0042500405,,1,8:30 pm ET,1,2026-06-13T00:00:00Z,1900-01-01T20:30:00Z,...,None,NaN,None,None,None,None,None,NaN,None,None
1314,00,2025-26,06/16/2026 00:00:00,0042500406,,1,8:30 pm ET,1,2026-06-16T00:00:00Z,1900-01-01T20:30:00Z,...,None,NaN,None,None,None,None,None,NaN,None,None


In [22]:
from nba_api.stats.endpoints import teamgamelogs
Usg25 = teamgamelogs.TeamGameLogs(
    league_id_nullable="00",
    season_nullable="2025-26",
    season_type_nullable="Regular Season",
    measure_type_player_game_logs_nullable='Scoring'
).get_data_frames()[0]

Usg25

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PCT_FGA_2PT,...,PCT_PTS_FT_RANK,PCT_PTS_OFF_TOV_RANK,PCT_PTS_PAINT_RANK,PCT_AST_2PM_RANK,PCT_UAST_2PM_RANK,PCT_AST_3PM_RANK,PCT_UAST_3PM_RANK,PCT_AST_FGM_RANK,PCT_UAST_FGM_RANK,AVAILABLE_FLAG
0,2025-26,1610612760,OKC,Oklahoma City Thunder,0022500625,2026-01-21T00:00:00,OKC @ MIL,W,48.0,0.600,...,1241,569,362,1166,132,1147,145,1223,87,1
1,2025-26,1610612740,NOP,New Orleans Pelicans,0022500624,2026-01-21T00:00:00,NOP vs. DET,L,48.0,0.628,...,93,57,180,1006,293,682,569,1144,164,1
2,2025-26,1610612738,BOS,Boston Celtics,0022500621,2026-01-21T00:00:00,BOS vs. IND,W,48.0,0.648,...,331,832,809,1166,132,1,1053,986,312,1
3,2025-26,1610612739,CLE,Cleveland Cavaliers,0022500620,2026-01-21T00:00:00,CLE @ CHA,W,48.0,0.540,...,349,187,310,435,866,1252,54,919,384,1
4,2025-26,1610612763,MEM,Memphis Grizzlies,0022500623,2026-01-21T00:00:00,MEM vs. ATL,L,48.0,0.624,...,301,791,362,989,314,1,1053,832,473,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,2025-26,1610612762,UTA,Utah Jazz,0022500087,2025-10-22T00:00:00,UTA vs. LAC,W,48.0,0.494,...,1003,516,502,65,1245,296,997,46,1262,1
1306,2025-26,1610612745,HOU,Houston Rockets,0022500001,2025-10-21T00:00:00,HOU @ OKC,L,58.0,0.598,...,181,1124,1074,1217,88,1,1053,1118,189,1
1307,2025-26,1610612760,OKC,Oklahoma City Thunder,0022500001,2025-10-21T00:00:00,OKC vs. HOU,W,58.0,0.500,...,652,134,824,573,726,752,517,674,630,1
1308,2025-26,1610612747,LAL,Los Angeles Lakers,0022500002,2025-10-21T00:00:00,LAL vs. GSW,L,48.0,0.584,...,695,197,56,918,386,613,652,1079,223,1
